**Пошаговое создание простого llm-агента**

Начнем с создания чат-агент без инструментов:

In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

load_dotenv()

/Users/alina/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
# создаем коннектор
llm = ChatOpenAI(
    model=os.environ.get("BASE_MODEL"),
    base_url=os.environ.get("BASE_URL"),
    api_key=os.environ.get("LLM_API_KEY"),
    temperature=0.2,
)

def chat(user_input: str):
    response = llm.invoke(user_input)
    return response.content

# проверим, что все работает
answer = chat("What is Python?")
print(answer)


Python is a **high-level, interpreted, general-purpose programming language**. It's renowned for its readability and simplicity, making it a popular choice for beginners and experienced developers alike.

Here's a breakdown of what that means and why it's so widely used:

**Key Characteristics:**

*   **High-level:** This means you don't have to worry about low-level details like memory management or CPU architecture. Python handles these complexities for you, allowing you to focus on solving problems with more abstract and human-readable code.

*   **Interpreted:** Unlike compiled languages (like C++ or Java), Python code is executed line by line by an interpreter at runtime. This makes the development cycle faster as you don't need to compile your code before running it.

*   **General-purpose:** Python isn't designed for one specific task. It can be used for a vast array of applications across many domains.

*   **Object-Oriented:** Python supports object-oriented programming (OOP) 

In [3]:
os.environ.get("BASE_MODEL")

'google/gemini-2.5-flash'

Добавим SystemMessage и HumanMessage. Ранее наша строка автоматически конвертировалась в HumanMessage при вызове метода invoke.

In [4]:
from langchain_core.messages  import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="You are a biologist and herpetologist. Give your answers exclusively as a biologist. You dont know nothing about programming!"),
    HumanMessage(content="What is Python?")
]
response = llm.invoke(messages)
print(response.content)


Python, my friend, is a fascinating genus of nonvenomous snakes belonging to the family Pythonidae. These magnificent reptiles are found in Africa, Asia, and Australia, inhabiting a diverse range of environments from lush rainforests to arid savannas.

They're constrictors, you see, meaning they subdue their prey by coiling around it and squeezing until the animal can no longer breathe. Their diet is quite varied, depending on the species and their size, ranging from small rodents and birds to larger mammals like deer and even crocodiles in some of the bigger species!

One of the most striking features of pythons is their sheer size. Some species, like the reticulated python, can grow to be among the longest snakes in the world, exceeding 20 feet in length! Others, like the ball python, are much smaller and more docile, making them popular in the pet trade – though I always advocate for responsible ownership and understanding their complex needs.

They're also known for their beautiful

Теперь добавим Structured output, чтобы получать структурированный ответ в виде описанного нами класса.

In [7]:
from pydantic import BaseModel, Field

# пишем структуру для ответа ЛЛМ
class ContactInfo(BaseModel):
    name: str = Field(description="Person's name")
    phone: str = Field(description="Phone number")
    date: str = Field(description="Date in YYYY-MM-DD format")

structured_llm = llm.with_structured_output(ContactInfo)

# prompt = (
#     "Преобразуй данные в JSON строго по схеме:\n"
#     "Алина Б., +79935555555, дата обращения - 15 ноября 2025"
# )
prompt = (
    "Алина Б., +79935555555, дата обращения - 15 ноября 2025"
)
result = structured_llm.invoke(prompt)

print(result.name)  
print(result.phone)  
print(result.date)  


Алина Б.
+79935555555
2025-11-15


**Как вообще это работает?**

Полученная JSON-строка передается в Pydantic модель через метод model_validate_json(), который:​
- Парсит JSON-строку
- Проверяет соответствие типам
- Приводит данные к нужным типам (если возможно)
- Выбрасывает ValidationError при ошибках
- Возвращает типизированный Python-объект

In [8]:
json_response = '{"name": "Алина Б.", "phone": "+79991234567", "date": "2025-11-11"}'

user = ContactInfo.model_validate_json(json_response)

print(user.name)  
print(type(user))  # <class 'ContactInfo'>


Алина Б.
<class '__main__.ContactInfo'>


Специально передадим неверный тип данных:

In [9]:
json_response = '{"name": "Алина Б.", "phone": 79991234567, "date": "2025-11-11"}'

user = ContactInfo.model_validate_json(json_response)

print(user.name)  
print(type(user))  # <class 'ContactInfo'>

ValidationError: 1 validation error for ContactInfo
phone
  Input should be a valid string [type=string_type, input_value=79991234567, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

Добавим React агента.

In [10]:
from langchain_core.tools import tool

@tool
def calculate(expression: str) -> str:
    """Calculate a mathematical expression. For example: '2 + 2' or '10 * 5'."""
    try:
        result = eval(expression)
        return f"Результат: {result}"
    except Exception as e:
        return f"Ошибка вычисления: {str(e)}"

@tool
def search_database(query: str) -> str:
    """Searching for information in the database."""
    db = {
        "Python": "Язык программирования высокого уровня",
        "LangChain": "Фреймворк для создания приложений с LLM",
        "ReAct": "Паттерн для агентов: Reasoning + Acting"
    }
    return db.get(query, f"Информация о '{query}' не найдена")

tools = [calculate, search_database]

In [11]:
from langgraph.prebuilt import create_react_agent

system_message = SystemMessage(
    content="You're a helpful assistant. Try using the tools to solve the problem! Think!"
)
agent = create_react_agent(
    llm, 
    tools, 
    prompt=system_message
)

/var/folders/6l/78d_4bx52ks02t3zdpykc0m40000gn/T/ipykernel_41084/1717792725.py:6: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [12]:
result = agent.invoke({"messages": [("user", "10+8*8")]})

In [13]:
len(result['messages'])

4

In [15]:
result['messages']

[HumanMessage(content='10+8*8', additional_kwargs={}, response_metadata={}, id='9317a0cf-6e19-4eee-9771-e0a6f7195dcd'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 65, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 56, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'cache_write_tokens': 0, 'video_tokens': 0}, 'cost': 0.0001795, 'is_byok': False, 'cost_details': {'upstream_inference_cost': 0.0001795, 'upstream_inference_prompt_cost': 1.95e-05, 'upstream_inference_completions_cost': 0.00016}}, 'model_provider': 'openai', 'model_name': 'google/gemini-2.5-flash', 'system_fingerprint': None, 'id': 'gen-1771405769-qWyR15ZzQ07j6g4PXgRP', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c7002-f7a3-7290-b6fe-48bcd4774288-0', tool_call